<a href="https://colab.research.google.com/github/JISHNU-SUNEESH/Generative_AI/blob/main/rag_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain --quiet
!pip install langchain_community --quiet
!pip install langchain_mistralai --quiet
!pip install langchain_huggingface --quiet
!pip install --upgrade --quiet langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 3.4 MB/s eta 0:00:00


In [20]:
from typing import Sequence
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_mistralai import ChatMistralAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains import create_history_aware_retriever,create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import BaseMessage,HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START,StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from typing_extensions import Annotated,TypedDict
import bs4

In [5]:
from google.colab import userdata
api_key=userdata.get('MISTRAL_AI_API_KEY')

In [18]:
llm=ChatMistralAI(model_name='mistral-large-latest',api_key=api_key)
loader=WebBaseLoader(web_paths=('https://lilianweng.github.io/posts/2023-06-23-agent/',),
                        bs_kwargs=dict(
                            parse_only=bs4.SoupStrainer(
                                class_=("post-content", "post-title", "post-header")
                            )
                        )
                        )

docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
vectorstore=InMemoryVectorStore.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)
history_aware_retiriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)


In [23]:
system_prompt=(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),

    ]
)

qa_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retiriever,qa_chain)

In [36]:
class State(TypedDict):
  input:int
  chat_history:Annotated[Sequence[BaseException],add_messages]
  context:str
  answer:str

def call_model(state:State):
  response=rag_chain.invoke(state)
  return{
      "chat_history":[
          HumanMessage(state["input"]),
          AIMessage(response["answer"]),
          ],
      "context":response["context"],
      "answer":response["answer"],

  }

workflow=StateGraph(state_schema=State)
workflow.add_edge(START,"model")
workflow.add_node("model",call_model)
memory=MemorySaver()
app=workflow.compile(checkpointer=memory)

In [25]:
config={"configurable":{"thread_id":"abc123"}}

result=app.invoke(
    {"input":"what is Task Decomposition?"},
    config=config,
)
print(result["answer"])

Task Decomposition is the process of breaking down a complex task into smaller, manageable steps. It can be achieved using techniques like Chain of Thought (CoT), where the model thinks through each step, or Tree of Thoughts, which explores multiple reasoning paths. This can be done by large language models (LLMs) using simple prompts, task-specific instructions, or human inputs.


In [26]:
print(result["chat_history"])

[HumanMessage(content='what is Task Decomposition?', additional_kwargs={}, response_metadata={}, id='9e031f12-4e09-48cf-ad13-b147c136c7eb'), AIMessage(content='Task Decomposition is the process of breaking down a complex task into smaller, manageable steps. It can be achieved using techniques like Chain of Thought (CoT), where the model thinks through each step, or Tree of Thoughts, which explores multiple reasoning paths. This can be done by large language models (LLMs) using simple prompts, task-specific instructions, or human inputs.', additional_kwargs={}, response_metadata={}, id='ea63dac8-78ef-4639-9779-05ba565831f4')]


In [29]:
import time

time.sleep(30)
result_2=app.invoke(
    {"input":"What is the one way of doing it?"},
    config=config
)
print(result_2["answer"])

One way of doing task decomposition is by using simple prompts with a Large Language Model (LLM). For example, you can use prompts like "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?" to break down a complex task into smaller, manageable steps. This method helps in planning and executing tasks more efficiently.


Agent:








In [34]:
from langchain.tools.retriever import create_retriever_tool

tool=create_retriever_tool(retriever,'blog_post_retriever','Searches and returns excerpts from the Autonomous Agents blog post.')

tools=[tool]

In [33]:
from langgraph.prebuilt import create_react_agent
agent_executor=create_react_agent(llm,tools,checkpointer=memory)

In [35]:
query = "What according to the blog post are common ways of doing it? redo the search"

for event in agent_executor.stream(
    {"messages":[HumanMessage(content=query)]},
    config=config,
    stream_mode="values"
):
  event["messages"][-1].pretty_print()

================================ Human Message =================================

What according to the blog post are common ways of doing it? redo the search
================================== Ai Message ==================================
Tool Calls:
  blog_post_retriever (1Q3czG6WQ)
 Call ID: 1Q3czG6WQ
  Args:
    query: What according to the blog post are common ways of doing it?
================================= Tool Message =================================
Name: blog_post_retriever

"content": "You will get instructions for code to write.\nYou will write a very long answer. Make sure that every detail of the architecture is, in the end, implemented as code.\nMake sure that every detail of the architecture is, in the end, implemented as code.\n\nThink step by step and reason yourself to the right decisions to make sure we get it right.\nYou will first lay out the names of the core classes, functions, methods that will be necessary, as well as a quick comment on their purpose.\n\nT